In [ ]:
%%writefile bubble.cpp

#include<omp.h>
#include<bits/stdc++.h>
using namespace std;

template<typename T>
void print(T arr) {
	for(auto& a: arr) {
		cout << a << ' ';
	}
	cout << '\n';
}

int main() {
	srand(time(NULL));
	double start, end;

    int n = 1000;
	vector<int> a(n);
	for(auto& e: a) {
		e = rand()%n;
	}
	vector<int> b = a;
	cout << "Input: ";
	/* print(a); */

    start = omp_get_wtime();
    for(int i = 0; i < n; i++) {
		#pragma omp parallel for
        for(int j = 0; j < n-1; j++) {
            if(a[j] > a[j+1]) {
              swap(a[j], a[j+1]);
			}
        }
    }
    end = omp_get_wtime();

	cout << "Output(Parallel): ";
	/* print(a); */
	cout << "Time taken(parallel): " << end - start << endl;


    start = omp_get_wtime();
	for(int i = 0; i < n; i++) {
        for(int j = 0; j < n-1; j++) {
            if(b[j] > b[j+1]) {
              swap(b[j], b[j+1]);
			}
        }
    }
    end = omp_get_wtime();
	cout << '\n';
	cout << "Output(Serial): ";
	/* print(b); */
	cout << "Time taken(Serial): " << end - start << endl;
    return 0;
}


Overwriting bubble.cpp


In [ ]:
%%script bash

g++ -fopenmp bubble.cpp -o bubble

In [ ]:
!./bubble

Input: Output(Parallel): Time taken(parallel): 0.0136272

Output(Serial): Time taken(Serial): 0.0110918


In [ ]:
%%writefile merge.cpp

#include<bits/stdc++.h>
#include<omp.h>

using namespace std;

template<typename T>
void print(T& a) {
	for(auto& e: a) {
		cout << e << ' ';
	}
	cout << '\n';
}

void merge(vector<int>& arr, int low, int mid, int high) {
	vector<int> a;
	vector<int> b;
	for(int i = low; i <= mid; i++) {
		a.push_back(arr[i]);
	}
	for(int i = mid+1; i <= high; i++) {
		b.push_back(arr[i]);
	}
	int n = a.size();
	int m = b.size();
	int i = 0, j = 0;
	int start = low;
	while(i!=n && j !=m) {
		if(a[i] <= b[j]) {
			arr[start++] = a[i++];
		} else {
			arr[start++] = b[j++];
		}
	}
	while(i < n) {
		arr[start++] = a[i++];
	}
	while(j < m) {
		arr[start++] = b[j++];
	}
}

void merge_sort(vector<int>& a, int low, int high) {
	if(high - low >= 1) {
		int mid = low + (high-low)/2;
		merge_sort(a, low, mid);
		merge_sort(a, mid+1, high);
		merge(a, low, mid, high);
	}
}

void mergeSort(vector<int>& a, int low, int high) {
	if(high - low >= 1) {
		int mid = low + (high-low)/2;
		#pragma omp parallel sections
		{
			#pragma omp section
			mergeSort(a, low, mid);
			#pragma omp section
			mergeSort(a, mid+1, high);
		}
		merge(a, low, mid, high);
	}
}

int main(int argc, char *argv[]) {
	srand(time(NULL));
    double start, end;

	int n = 100;
	vector<int> a(n);
	for(auto& e: a) {
		e = rand()%n;
	}
	vector<int> b = a;
	cout << "Input: ";
	print(a);

    start = omp_get_wtime();
	mergeSort(a, 0, n-1);
    end = omp_get_wtime();

	cout << "Output: ";
	print(a);
	cout << "Time taken(for parallel): " << end - start << endl;

    start = omp_get_wtime();
	merge_sort(b, 0, n-1);
	end = omp_get_wtime();
	cout << "Time taken(for serial): " << end - start << endl;

    return 0;
}


Overwriting merge.cpp


In [ ]:
%%script bash

g++ -fopenmp merge.cpp
./a.out

Input: 39 92 80 25 79 66 95 55 48 18 15 1 3 94 10 18 78 44 28 75 97 77 14 15 11 49 58 29 32 17 46 24 9 79 1 41 45 97 96 93 15 64 47 70 10 57 89 88 53 69 16 2 98 30 17 9 79 76 38 63 93 37 39 54 16 41 95 61 38 44 7 5 60 54 75 70 63 16 10 68 85 26 71 83 56 40 44 87 16 35 3 61 72 42 68 88 83 63 1 21 
Output: 1 1 1 2 3 3 5 7 9 9 10 10 10 11 14 15 15 15 16 16 16 16 17 17 18 18 21 24 25 26 28 29 30 32 35 37 38 38 39 39 40 41 41 42 44 44 44 45 46 47 48 49 53 54 54 55 56 57 58 60 61 61 63 63 63 64 66 68 68 69 70 70 71 72 75 75 76 77 78 79 79 79 80 83 83 85 87 88 88 89 92 93 93 94 95 95 96 97 97 98 
Time taken(for parallel): 0.000471276
Time taken(for serial): 0.000150263
